In [1]:
import pandas as pd
seed_value = 2043
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import random
random.seed(seed_value)

In [2]:
import numpy as np
class WOE_Encoder():
    def __init__(self, cols=None, size=None):
            self.cols = cols
            self.min_samples=1
            #self.bins=10000
            self.bins=int(size/10)
            self._mapping = {}
            
    def WOE_fit(self, X, y):
        for col in self.cols:
            X[col]=X[col].fillna(-9999)
            if (len(np.unique(X[col]))>100):
                binned_x = pd.qcut(X[col], self.bins,  duplicates='drop')
                d0 = pd.DataFrame({'x': binned_x, 'y':y})
            else:
                d0 = pd.DataFrame({'x': X[col], 'y': y})
            #print (d0)
            # Share of positive (resp. negative) labels for each category P(X=X_i | Y=1) (resp. P(X=X_i | Y=0))
            #mapping = y.groupby(X[col]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping = y.groupby(d0["x"]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping['neg'] = mapping['count'] - mapping['pos']
            mapping[['pos', 'neg']] /= mapping[['pos', 'neg']].sum()
            # For corner cases, defaulting to WOE = 0 (meaning no info). To avoid division by 0 we use default values.
            undef = (mapping['count'] < self.min_samples) | (mapping['pos'] == 0) | (mapping['neg'] == 0)
            mapping.loc[undef, ['pos', 'neg']] = -1
            # Final step, log of ratio of probabily estimates
            mapping['value'] = np.log((mapping['pos'] +0.0001)/ (mapping['neg']+0.0001))
            self._mapping[col] = mapping
            

        X_encoded = X.copy(deep=True)
        for col, mapping in self._mapping.items():
            X_encoded.loc[:, col] = X_encoded[col].fillna(-9999).map(mapping['value'])
            X_encoded[col].fillna(0, inplace=True)
             
        return X_encoded 

In [3]:
syn_df=pd.read_csv('C:/Users/IICT/JOY/Research-JOY/GAN/GAN-Privacy-JOY-v2/Colab-Data/Colab-data-7043/synthetic_data_GAN_mWOE_privacy_1_70persent_training_dataset7043_set4.csv')
y_syn_df=syn_df['churn']
X_syn_df=syn_df.drop(['churn'], axis=1)
Obj = WOE_Encoder(cols=X_syn_df.columns, size=X_syn_df.shape[0])
X_encoded = Obj.WOE_fit(X_syn_df, y_syn_df)
X_encoded.head() 

,Unnamed: 0,SeniorCitizen,tenure,MonthlyCharges,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges
0,0.56473,0.000000,0.564730,-0.370174,-1.115821,0.141686,0.141686,0.141686,0.141686,-0.370174,-0.370174,1.336967,0.564730,0.000000,0.952481,-1.115821,0.141686,-0.370174,-0.370174,-0.370174
1,0.56473,0.141686,0.564730,0.141686,-1.115821,-0.370174,-1.115821,0.141686,0.000000,0.141686,0.141686,0.952481,-0.370174,0.141686,0.141686,-1.115821,0.952481,0.000000,-1.115821,-0.370174
2,0.56473,-1.115821,-1.115821,0.564730,0.141686,-0.370174,-1.115821,-0.370174,-1.115821,-0.370174,0.141686,0.141686,0.141686,-1.115821,0.141686,-0.370174,0.141686,-0.370174,-0.370174,-0.370174
3,0.56473,0.564730,0.000000,0.564730,-0.370174,-1.115821,-1.115821,-0.370174,0.564730,-1.115821,-0.370174,0.000000,0.000000,0.564730,0.564730,0.564730,-1.115821,0.141686,0.141686,0.141686
4,0.56473,1.748546,2.233023,1.748546,1.336967,2.900164,0.564730,1.336967,1.336967,1.336967,2.233023,0.952481,1.748546,1.336967,1.336967,1.336967,0.564730,1.336967,1.748546,2.233023


In [4]:
org_test_df=pd.read_csv('C:/Users/IICT/JOY/Research-JOY/GAN/GAN-Privacy-JOY-v2/Colab-Data/Colab-data-7043/Original_data_GAN_mWOE_privacy_1_30percent_testing_dataset7043.csv')
y_test=org_test_df['churn']
X_test=org_test_df.drop(['churn'], axis=1)
Obj_test = WOE_Encoder(cols=X_test.columns, size=X_test.shape[0])
X_encoded_test = Obj_test.WOE_fit(X_test, y_test)
X_encoded_test.head() 

,Unnamed: 0,SeniorCitizen,tenure,MonthlyCharges,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges
0,0.569715,-0.155969,1.586662,-0.110356,-0.021324,-0.412361,0.229132,-0.033252,-0.033252,-0.482741,0.699000,0.610469,0.630512,0.728991,0.268615,0.367391,0.733324,0.331370,0.885785,0.967358
1,-1.172108,-0.155969,-1.191184,-1.455578,0.023052,0.318783,0.229132,0.003738,0.169368,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,0.733324,0.331370,-0.692514,-0.506478
2,0.967358,-0.155969,-1.233862,-0.259956,-0.021324,-0.412361,-0.696401,0.003738,-0.141946,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-2.375949,-0.650735,-0.422533,0.569715
3,0.136166,-0.155969,1.586662,0.967358,-0.021324,0.318783,0.229132,0.003738,-0.141946,0.722046,0.699000,0.610469,-0.269859,0.728991,0.268615,0.367391,0.733324,-0.650735,0.885785,-0.390795
4,0.136166,-0.155969,-1.593819,1.363492,0.023052,0.318783,0.229132,0.003738,-0.141946,-0.482741,0.699000,0.610469,0.630512,-0.831894,0.268615,0.367391,-2.375949,-0.650735,-0.692514,0.136166


In [5]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=6.579332246575682e-09)

classifier.fit(X_encoded,y_syn_df)


cv_method = RepeatedKFold(n_splits=10, 
                          n_repeats=3, 
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 5)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1375 164 192 382
pod:  0.6655052264808362
pof:  0.10656270305393112
AUC:  0.7794712617134526
recall:  0.6655052264808362
precision:  0.6996336996336996
F1-Score:  0.6821428571428572
accuracy:  0.8315191670610507


In [6]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l1', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)



1347 192 165 409
pod:  0.7125435540069687
pof:  0.12475633528265107
AUC:  0.7938936093621588
recall:  0.7125435540069687
precision:  0.6805324459234608
F1-Score:  0.6961702127659574
accuracy:  0.8310459062943681


In [7]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(bootstrap= True, max_depth= 80, max_features= 3, min_samples_leaf= 3, min_samples_split= 12, n_estimators= 1000)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1381 158 244 330
pod:  0.5749128919860628
pof:  0.10266406757634827
AUC:  0.7361244122048571
recall:  0.5749128919860628
precision:  0.6762295081967213
F1-Score:  0.6214689265536725
accuracy:  0.8097491717936584


In [8]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(metric='manhattan', weights='uniform', n_neighbors=19 )  
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1432 107 259 315
pod:  0.5487804878048781
pof:  0.06952566601689408
AUC:  0.739627410893992
recall:  0.5487804878048781
precision:  0.7464454976303317
F1-Score:  0.6325301204819278
accuracy:  0.8267865593942262


In [9]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

#classifier = DecisionTreeClassifier(criterion='entropy', splitter='best')
classifier = DecisionTreeClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1342 197 253 321
pod:  0.5592334494773519
pof:  0.12800519818063677
AUC:  0.7156141256483576
recall:  0.5592334494773519
precision:  0.6196911196911197
F1-Score:  0.5879120879120879
accuracy:  0.7870326549929011


In [10]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

#classifier = GradientBoostingClassifier(max_features=None, max_depth=3, criterion='friedman_mse')
classifier = GradientBoostingClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1418 121 271 303
pod:  0.5278745644599303
pof:  0.07862248213125406
AUC:  0.7246260411643382
recall:  0.5278745644599303
precision:  0.714622641509434
F1-Score:  0.6072144288577154
accuracy:  0.8144817794604827


In [11]:
#XGBClassifier
from xgboost import XGBClassifier
classifier =  XGBClassifier(random_state=0) 

classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X, y = y, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1416 123 271 303
pod:  0.5278745644599303
pof:  0.07992202729044834
AUC:  0.723976268584741
recall:  0.5278745644599303
precision:  0.7112676056338029
F1-Score:  0.6060000000000001
accuracy:  0.8135352579271179


In [12]:
#FNN
from keras.models import Sequential
from keras.layers import Dense
#create model
model = Sequential()
#get number of columns in training data
n_cols = X_encoded_test.shape[1]
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#compile model using mse as a measure of model performance
#model.compile(optimizer='adam', loss='mean_squared_error')
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  
from keras.callbacks import EarlyStopping
#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)
#train model

model.fit(X_encoded, y_syn_df, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
y_pred = model.predict(X_encoded_test)
y22_pred=y_pred.round()
from sklearn.metrics import confusion_matrix
 
tn, fp, fn, tp  = confusion_matrix(y_test, y22_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)    

Epoch 1/30
124/124 [==============================] - 2s 7ms/step - loss: 0.2629 - accuracy: 0.8935 - val_loss: 0.2024 - val_accuracy: 0.9270
Epoch 2/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1918 - accuracy: 0.9247 - val_loss: 0.1845 - val_accuracy: 0.9249
Epoch 3/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1709 - accuracy: 0.9341 - val_loss: 0.1633 - val_accuracy: 0.9361
Epoch 4/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1573 - accuracy: 0.9399 - val_loss: 0.1637 - val_accuracy: 0.9341
Epoch 5/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1543 - accuracy: 0.9419 - val_loss: 0.1502 - val_accuracy: 0.9452
Epoch 6/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1468 - accuracy: 0.9430 - val_loss: 0.1484 - val_accuracy: 0.9493
Epoch 7/30
124/124 [==============================] - 1s 5ms/step - loss: 0.1440 - accuracy: 0.9440 - val_loss: 0.1445 - val_accuracy: 0.9493
Epoch 